# Quickstart – gwrefpy <img src="https://github.com/user-attachments/assets/4763ec7a-c703-414f-ba81-1520793f5f8d" alt="gwrefpy" width="200" height="200" style="float: right; margin-left: px;" />

This quick introductory example demonstrates how to use `gwrefpy` to analyze an observation and a reference well, calibrate a model, save the project, and then reload and visualize the results.

We will go step by step:
1. Store the data in `Well` objects  
2. Build a `Model` and run calibration  
3. Save the project and reload it  
4. Plot the results  


In [ ]:

import logging
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import dates

from gwrefpy import Model, Well

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.WARNING)

## 1. Import groundwater level measurements 
Importing groundwaterlevel for a reference well and a observationswell as csv files and creates panda series.

In [ ]:
# Loading observation well data
obs_df = pd.read_csv(os.path.join(os.getcwd(), "examples/data/OBS_qst.csv"), parse_dates=["date"])
obs_series = pd.Series(obs_df["groundwaterlevel_m_a_s_l"].astype(float).values, index=obs_df["date"])

# Loading reference well data
ref_df = pd.read_csv("examples/data/REF_qst.csv", parse_dates=["date"])
ref_series = pd.Series(ref_df["groundwaterlevel_m_a_s_l"].astype(float).values, index=ref_df["date"])


## 2. Create well objects 💧

We now add our time series to `Well` objects.  The following boolean condition marks if the well is reference or observation. 
- `is_reference=False` for the observation well  
- `is_reference=True` for the reference well  

In [ ]:




obs = Well(name="Obs_well", is_reference=False)
obs.add_timeseries(obs.series)

ref = Well(name="Ref_well", is_reference=True)
ref.add_timeseries(ref.series)

## 3. Build a model and calibrate ⚙️

We create a `Model` and add both the reference well and the observation well.
Then we choose a calibration period – here we just use the whole time series.
The `model.fit` function takes the observation well (`obs`), the reference well (`ref`), the (`time_equivalent`), and the start and end dates of the calibration period. It checks how well the reference well values match the observation well values. 

In [ ]:
model = Model(name="Quickstart_model")
model.add_well(obs)
model.add_well(ref)

calibration_period_start = dates[0]
calibration_period_end = dates[-1]

model.fit(
    obs,
    ref,
    time_equivalent="0D",
    tmin=calibration_period_start,
    tmax=calibration_period_end,
)

## 4. Save and reload the project 💾

We save the model to a `.gwref` file and reload it again.  
This illustrates how projects can be shared and reused.

In [ ]:
project_path = "examples/quickstart_model"
model.save_project(project_path, overwrite=True)

# simulate a fresh session
del model, obs, ref

model_loaded = Model("quickstart_model_loaded")
model_loaded.open_project(f"{project_path}.gwref")

## 5. Plot the results 📈

Finally, we plot the calibrated model using the built-in plotting functionality.

In [ ]:
model_loaded.plot(plot_style="fancy", color_style="monochrome")
plt.show()
